In [79]:
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")

<h1 style="text-align:center">🏰 Data Modelling</h1> 

<h3 style="text-align:center">Predict BTC prices</h3> 

# 🎖 1. Get newest data points

## ➡️ Define symbol and time interval

In [80]:
symbol = "BTCUSDT"
PERIOD = "15m"

In [81]:
!jupyter nbconvert --to python ../obtain/get_newest_price.ipynb
!python ../obtain/get_newest_price.py

[NbConvertApp] Converting notebook ../obtain/get_newest_price.ipynb to python
[NbConvertApp] Writing 1547 bytes to ../obtain/get_newest_price.py


In [82]:
pd_df = pd.read_csv(f"../../datastore/price/{symbol}_{PERIOD}.csv")
pd_df = pd_df.iloc[-900:]
pd_df

,Kline open time,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
171632,1672437600000,16585.06,16597.00,16584.35,16588.54,1099.59680,1672438499999,1.824181e+07,31056,534.96619,8.874969e+06
171633,1672438500000,16589.03,16589.85,16577.57,16583.98,649.76779,1672439399999,1.077536e+07,19966,304.80541,5.054799e+06
171634,1672439400000,16583.98,16591.70,16578.62,16588.59,1119.28578,1672440299999,1.856348e+07,26707,558.41996,9.261704e+06
171635,1672440300000,16588.59,16592.48,16581.06,16583.27,1081.54287,1672441199999,1.793969e+07,28099,506.76685,8.406024e+06
171636,1672441200000,16582.81,16599.82,16568.00,16593.88,1710.26619,1672442099999,2.837227e+07,40712,852.56365,1.414426e+07
...,...,...,...,...,...,...,...,...,...,...,...
172527,1673243100000,17217.83,17219.64,17200.86,17206.33,780.20558,1673243999999,1.342800e+07,13217,382.02312,6.574877e+06
172528,1673244000000,17205.95,17220.59,17191.67,17201.00,2334.04674,1673244899999,4.015890e+07,52979,1240.30954,2.134104e+07
172529,1673244900000,17201.42,17210.35,17199.00,17210.01,1330.14318,1673245799999,2.288644e+07,41151,691.37532,1.189601e+07
172530,1673245800000,17209.58,17218.64,17203.04,17203.43,1333.28379,1673246699999,2.294574e+07,42014,689.32453,1.186355e+07


## ➡️ Select features

### 📌 Using Open time as index to visualize later

In [83]:
pd_df["Open price"] = pd_df["Open price"].apply(lambda price: float(price))
pd_df = pd_df.set_index("Kline open time").sort_index() 
pd_df.index = pd.to_datetime(pd_df.index, unit='ms') + pd.Timedelta('07:00:00')
pd_df

,Open price,High price,Low price,Close price,Volume,Kline Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
Kline open time,,,,,,,,,,
2022-12-31 05:00:00,16585.06,16597.00,16584.35,16588.54,1099.59680,1672438499999,1.824181e+07,31056,534.96619,8.874969e+06
2022-12-31 05:15:00,16589.03,16589.85,16577.57,16583.98,649.76779,1672439399999,1.077536e+07,19966,304.80541,5.054799e+06
2022-12-31 05:30:00,16583.98,16591.70,16578.62,16588.59,1119.28578,1672440299999,1.856348e+07,26707,558.41996,9.261704e+06
2022-12-31 05:45:00,16588.59,16592.48,16581.06,16583.27,1081.54287,1672441199999,1.793969e+07,28099,506.76685,8.406024e+06
2022-12-31 06:00:00,16582.81,16599.82,16568.00,16593.88,1710.26619,1672442099999,2.837227e+07,40712,852.56365,1.414426e+07
...,...,...,...,...,...,...,...,...,...,...
2023-01-09 12:45:00,17217.83,17219.64,17200.86,17206.33,780.20558,1673243999999,1.342800e+07,13217,382.02312,6.574877e+06
2023-01-09 13:00:00,17205.95,17220.59,17191.67,17201.00,2334.04674,1673244899999,4.015890e+07,52979,1240.30954,2.134104e+07
2023-01-09 13:15:00,17201.42,17210.35,17199.00,17210.01,1330.14318,1673245799999,2.288644e+07,41151,691.37532,1.189601e+07


### 📌 Visualize prices as candle stick

In [84]:
fig = go.Figure(data=[go.Candlestick(x=pd_df.index,
                open=pd_df['Open price'],
                high=pd_df['High price'],
                low=pd_df['Low price'],
                close=pd_df['Close price'])])
# set new height and width
fig.update_layout(
    height=800,
    width=1000,
    title_text="BTC/USDT price",
    yaxis_title="Price (BTC/USDT)",
    xaxis_title="Date",
    xaxis_rangeslider_visible=True
)

fig.show()

### 📌 Using open price as feature and target as well

In [85]:
dataset = pd_df.filter(["Open price"]).values
dataset[:5]

array([[16585.06],
       [16589.03],
       [16583.98],
       [16588.59],
       [16582.81]])

# 🎖 2. Prepare train-test set

✅ Train-Test ratio: `80%` train, `20%` test <br>
✅ Train-Valid ratio: `70%` train, `30%` valid

In [86]:
TRAIN_TEST_LENGTH = int(len(dataset) * 0.8)
TRAIN_VALID_LENGTH = int(TRAIN_TEST_LENGTH * 0.7)

## ➡️ Scale data

In [87]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(dataset)
scaled_data[:5]

array([[0.11230763],
       [0.11771922],
       [0.11083546],
       [0.11711945],
       [0.1092406 ]])

## ➡️ Prepare time series data

In [88]:
def get_split_data(start, end = None):
    data = scaled_data[start:end]
    X = []
    Y = []
    for i in range(60,len(data)):
        X.append(data[i-60:i,:])
        Y.append(data[i,:])

    X, Y = np.array(X), np.array(Y)
    return X,Y

## ➡️ Split train-test set

In [89]:
X_origin_train, Y_origin_train = get_split_data(0, TRAIN_TEST_LENGTH)
X_test, Y_test = get_split_data(TRAIN_TEST_LENGTH-60, )

print(X_origin_train.shape, Y_origin_train.shape)
print(X_test.shape, Y_test.shape)

(660, 60, 1) (660, 1)
(180, 60, 1) (180, 1)


## ➡️ Split train-valid set

In [90]:
X_train, Y_train = get_split_data(0, TRAIN_VALID_LENGTH)
X_valid, Y_valid = get_split_data(TRAIN_VALID_LENGTH-60, TRAIN_TEST_LENGTH)

print(X_train.shape, Y_train.shape)
print(X_valid.shape, Y_valid.shape)

(443, 60, 1) (443, 1)
(217, 60, 1) (217, 1)


# 🎖 3. Naive training

## ➡️ Visualize predictions

In [91]:
def visualize_prediction(preds):
    data = pd_df[["Open price"]]
    train = data[:TRAIN_VALID_LENGTH]
    valid = data[TRAIN_VALID_LENGTH:TRAIN_TEST_LENGTH].reset_index()
    valid["Predict"] = preds[:, 0]
    valid = valid.set_index("Kline open time")
    valid["Predict"]= valid["Predict"].apply(lambda price: float(price))

    concat_df = pd.concat([train, valid], axis=0)
    fig = px.line(concat_df[["Open price","Predict"]], title="BTC/USDT price" , width=1000, height=800)
    fig.show()

## ➡️ Linear Regression

### 📌 Build model and train

## ➡️ Linear Regression

### 📌 Build model and train

In [92]:
from sklearn.linear_model import LinearRegression

In [93]:
model = LinearRegression()
model.fit(X_train.reshape(X_train.shape[0], X_train.shape[1]), Y_train)

LinearRegression()

### 📌 Predict and transform to the original scale

In [94]:
predictions = model.predict(X_valid.reshape(X_valid.shape[0], X_valid.shape[1]))
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
predictions.shape

(217, 1)

### 📌 Visualize the prediction

In [95]:
visualize_prediction(predictions)

## ➡️ Gated Recurrent Unit (GRU)

### 📌 Build model and train

In [96]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

In [97]:
model = Sequential()
model.add(GRU(200, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(GRU(100))
model.add(Dense(X_train.shape[2]))
model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X_train, Y_train, batch_size=1, epochs=1)

443/443 [==============================] - 9s 15ms/step - loss: 0.0017


### 📌 Predict and transform to the original scale

In [98]:
predictions = model.predict(X_valid)
predictions = scaler.inverse_transform(predictions)
predictions.shape

7/7 [==============================] - 1s 15ms/step


(217, 1)

### 📌 Visualize the prediction

In [99]:
visualize_prediction(predictions)

## ➡️ Seasonal Auto Regressive Integrated Moving Average (SARIMAX)

In [23]:
from statsmodels.tsa.api import SARIMAX

### 📌 Build model and train

In [ ]:
predictions = list()
for t in range(X_valid.shape[0]):
    model = SARIMAX(X_valid[t])
    model_fit = model.fit()
    output = model_fit.forecast()
    pred_price = output[0]
    predictions.append(pred_price)

### 📌 Predict and transform to the original scale

In [25]:
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
predictions.shape

(217, 1)

### 📌 Visualize the prediction

In [26]:
visualize_prediction(predictions)        

# 🎖 4. Model selection and evaluation

In [27]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, r2_score
from estimator import LSTMEstimator, SarimaxEstimator, LinearRegressionEstimator

## ➡️ Naive cross validation to select model

### 📌 Cross validation method

In [28]:
tscv = TimeSeriesSplit(n_splits=3)

- The method that can be used for cross-validating the time-series model is cross-validation on a rolling basis. Start with a small subset of data for training purpose, forecast for the later data points and then checking the accuracy for the forecasted data points. The same forecasted data points are then included as part of the next training dataset and subsequent data points are forecasted. <br>
![](https://miro.medium.com/max/640/1*XcqvKVTQ6U_zszSD52lSqA.webp)

### 📌 Metric method

In [29]:
metric = make_scorer(r2_score)

- $R^2$ score is used to evaluate the performance of a linear regression model. It is the amount of the variation in the output dependent attribute which is predictable from the input independent variable(s) <br>
![](https://vitalflux.com/wp-content/uploads/2019/07/R-squared-formula-function-of-SSE-and-SST.jpg)

### 📌 Evaluate models

In [30]:
estimators = [
    LinearRegressionEstimator(),
    LSTMEstimator(), 
    SarimaxEstimator(), 
]

In [ ]:
scores_dict = {}

for estimator in estimators:
    scores = cross_val_score(estimator, X_origin_train, Y_origin_train, scoring=metric, cv=tscv, n_jobs=-1)
    print(estimator.__class__.__name__, scores.mean())
    scores_dict[estimator.__class__.__name__] = scores
    
scores_df = pd.DataFrame(scores_dict)
scores_df.index = scores_df.index.map(lambda x: f'fold_{x+1}')
scores_df.loc['mean'] = scores_df.mean()

### 📌 Score and reveal best model

In [32]:
scores_df

,LinearRegressionEstimator,LSTMEstimator,SarimaxEstimator
fold_1,0.700039,0.287115,0.686077
fold_2,0.862796,0.687337,0.807752
fold_3,0.942435,0.821581,0.914218
mean,0.835090,0.598678,0.802682


- The best best_estimator is one that have highest mean of scores

In [33]:
best_estimator = scores_df.loc['mean'].idxmin()
best_estimator

'LSTMEstimator'

## ➡️ Select best model with best hyperparameters using grid search

In [34]:
from estimator import Estimator, LinearRegressionEstimator, SarimaxEstimator, LSTMEstimator 
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline

### 📌 Cross validation and metric method

- Using aforementioned methods

In [35]:
tscv = TimeSeriesSplit()
metric = make_scorer(r2_score)

### 📌 Evaluate hyperparameters and models 

In [36]:
parameters = [
    {
        'reg__estimator': [LinearRegressionEstimator()]
    },
    {
        'reg__estimator': [SarimaxEstimator()],
        'reg__estimator__order': [(1, 1, 1), (3, 1, 1)],
        'reg__estimator__seasonal_order': [(1, 1, 1, 12), (3, 1, 1, 12)]
    },
    {
        'reg__estimator': [LSTMEstimator()],
        'reg__estimator__epochs': [1, 5, 10],
        'reg__estimator__batch_size': [4, 16, 32],
        'reg__estimator__neurons': [50, 100, 200]
    },
]

In [ ]:
pipeline = Pipeline(
    steps=[("reg", Estimator())]
)

grid_search = GridSearchCV(pipeline, parameters, scoring=metric, cv=tscv, n_jobs=2, verbose=0)
grid_search.fit(X_train, Y_train)

- Best model with best hyperparameters

In [38]:
grid_search.best_params_

{'reg__estimator': SarimaxEstimator(order=(1, 1, 1)),
 'reg__estimator__order': (1, 1, 1),
 'reg__estimator__seasonal_order': (1, 1, 1, 12)}

# 🎖 5. Predict future prices

- We utilize the best model to predict prices

In [ ]:
from statsmodels.tsa.api import SARIMAX

data = pd_df.filter(["Open price"])

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data).reshape(-1)

predictions = list()
model = SARIMAX(scaled_data, order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit()
output = model_fit.forecast()
pred_price = output[0]
predictions.append(pred_price)

In [40]:
pred_price = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
pred_price

array([[17218.5754658]])